In [1]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import os
from pathlib import Path
import re
import pickle
# ver qtos o modelo apenas de ner acertaria
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk    
from nltk import tokenize 
import torch
from transformers import BertTokenizer,BertForTokenClassification
import numpy as np
import json   
from importlib import reload  # Python 3.4+
import random
import model as mod
from model import BertForChunkClassification
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup

from torch.utils.data import DataLoader
from importlib import reload 
#from eval import predict
import eval
#import importlib
#importlib.reload(module)
import dataset
from dataset import InputFeatures, load_and_cache_examples
import functionsAval as f
from sklearn_crfsuite.metrics import flat_f1_score, flat_classification_report
import joblib
import os


In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
def has_numbers(inputString):
    return str(bool(re.search(r'\d', inputString)))

print(has_numbers('metionina 5mg'))

def isstopword(p):
  return str(p in stopwords.words('portuguese'))

print(isstopword('pelo'))


True
True


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lisat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [53]:
def powerset(entidade):    
    lista=list()
    for i in range(1, len(entidade)+1):
        #lista.append(entidade[i-1])
        for j in range(len(entidade) - i + 1):
            lista.append(entidade[j:j + i])
    return lista
        
#print(list(powerset([4, 5, 6])))
print(powerset(['dor', 'no', 'peito']))


dicPostagger = f.load_obj('dic_postagger')
def tipoPostaggerTokens(token, dicPostagger):
    postagger = 'N' # na duvida é N
    if token.lower() in dicPostagger.keys():
        postagger = dicPostagger.get(token.lower())
    return postagger
tipoPostaggerTokens('coração', dicPostagger)

[['dor'], ['no'], ['peito'], ['dor', 'no'], ['no', 'peito'], ['dor', 'no', 'peito']]


'N'

In [4]:
f = reload(f)
reload(dataset)
reload(eval)
reload(mod)

# em numero de frases
BATCH=506
#BATCH=5
#BATCH=800
#BATCH=8000 
print('BATCH:', BATCH)

BATCH: 506


In [5]:
dicSentences_new_test = f.loadSentencesTest()
print(len(dicSentences_new_test))
dicSentences_new_test = {k: v for k, v in dicSentences_new_test.items() if k<BATCH}
print(dicSentences_new_test[0])
#print(dicSentences_new_test[27])
print('numero de sentencas no total:', len(dicSentences_new_test))

sentences=list()
for key, value in dicSentences_new_test.items():
    if key<BATCH:
        tokens = value[0]
        tokens = [tok[0] for tok in tokens]
        sentences.append(' '.join(tokens).strip())
#print(sentences[0])

tags, tokens = f.predictBERTNER_IO(sentences, 'all')
dic_predictions = f.getDicPredictions(tags, tokens)
dic_predictions_ner_bert = f.getDicPredictions(tags, tokens)
print(dic_predictions[0])
print('len(dic_predictions):', len(dic_predictions))
#print(dic_predictions[9])
f.save_obj('dic_predictions_results_ner_'+str(BATCH), dic_predictions)
#dic_predictions = f.load_obj('dic_predictions_results_ner_'+str(BATCH))

print('len(dicSentences_new_test):', len(dicSentences_new_test))
print('len(dic_predictions):', len(dic_predictions))

region_true_list, region_pred_list, lista_erros = f.getListaRegionsTruePred(BATCH, dicSentences_new_test, dic_predictions)
f.save_obj('region_true_list'+str(BATCH), region_true_list)
print('region_pred_list[:4]:', region_pred_list[:4])
print('region_true_list[:4]:', region_true_list[:4])
print('lista_erros[:8]:', lista_erros[:8])
print('len(lista_erros):', len(lista_erros))
print('len(set(lista_erros)):', len(set(lista_erros)))
print('len(region_true_list):', len(region_true_list))
print('len(region_pred_list):', len(region_pred_list))

print('-----Avaliando nested só com modelo de NER com BERT:-----')

print(classification_report(region_true_list, region_pred_list, digits=6))
print(confusion_matrix(region_true_list, region_pred_list))


Pegando sentencas de teste gabarito: dic_sentencesTest.pkl
506
[[['Lucas', 0, 43], [',', 1, 48], ['74', 2, 50], ['anos', 3, 53], ['.', 4, 57]], []]
numero de sentencas no total: 506
idx2tag: {0: 'Teste', 1: 'Anatomia', 2: 'O', 3: 'Problema', 4: 'Tratamento', 5: '<pad>'}
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
len(dic_predictions): 506
len(dicSentences_new_test): 506
len(dic_predictions): 506
region_pred_list[:4]: ['Problema', 'Tratamento', 'Problema', 'Problema']
region_true_list[:4]: ['Problema', 'Tratamento', 'Problema', 'Problema']
lista_erros[:8]: [7, 8, 13, 13, 14, 15, 15, 15]
len(lista_erros): 383
len(set(lista_erros)): 169
len(region_true_list): 1140
len(region_pred_list): 1140
-----Avaliando nested só com modelo de NER com BERT:-----
              precision    recall  f1-score   support

    Anatomia   0.829268  0.346939  0.489209       196
           O   0.000000  0.000000  0.000000       149
    Problema   0.798319  0.843195  0.820144       338
      

In [6]:
tags[0]

['O', 'O', 'O', 'O', 'O']

In [7]:
tokens[0]

['Lucas', ',', '74', 'anos', '.']

In [11]:
dic_predictions_ner_bert[42]

[[['Paciente', 0],
  ['em', 1],
  ['tratamento', 2],
  ['para', 3],
  ['ICC', 4],
  ['diastólica', 5],
  ['há', 6],
  ['cerca', 7],
  ['de', 8],
  ['5', 9],
  ['a', 10],
  ['com', 11],
  ['melhora', 12],
  ['importante', 13],
  ['dos', 14],
  ['sintomas', 15],
  ['após', 16],
  ['início', 17],
  ['do', 18],
  ['tratamento', 19],
  ['.', 20]],
 [['ICC diastólica', [4, 5], 'Problema'], ['tratamento', [19], 'Tratamento']]]

In [13]:
# com CRF

#JANELA=4
#CLUSTER=300

pathTest=r'C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\preProcessamento\data-ner/nested_test.conll'

with open(pathTest, encoding='utf-8') as fi:
  testdata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in fi.read().split('\n\n')]


def read_clusters(cluster_file):
    word2cluster = {}
    try:
        with open(cluster_file, encoding='utf-8') as i:
            for num, line in enumerate(i):
                if line:
                    word, cluster = line.strip().split('\t')
                    word2cluster[word] = cluster
    except:
        raise
    return word2cluster

def word2features(sent, i):
  #if str(sent[i][0])!='nan':#esto es extra
    word = sent[i][0]
    postag = tipoPostaggerTokens(word, dicPostagger)
    cluster = word2cluster[word.lower()] if word.lower() in word2cluster else "0"
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[:3]': word[:3],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'word.cluster': cluster
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:word.cluster': cluster
        })
    else:
        features['BOS'] = True
    
    if i > 1 and (JANELA==2 or JANELA==3 or JANELA==4):
        word1 = sent[i-2][0]
        postag1 = sent[i-2][1]
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-2:word.lower()': word1.lower(),
            '-2:word.istitle()': word1.istitle(),
            '-2:word.isupper()': word1.isupper(),
            '-2:postag': postag1,
            '-2:word.cluster': cluster
        })
    else:
        features['Second_word'] = True

    if i > 2 and (JANELA==3 or JANELA==4):
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-3:word.lower()': word1.lower(),
            '-3:word.istitle()': word1.istitle(),
            '-3:word.isupper()': word1.isupper(),
            '-3:postag': postag1,
            '-3:word.cluster': cluster
        })
    else:
        features['Third_word'] = True
    
    if i > 3 and JANELA==4:
        word1 = sent[i-2][0]
        postag1 = sent[i-2][1]
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-4:word.lower()': word1.lower(),
            '-4:word.istitle()': word1.istitle(),
            '-4:word.isupper()': word1.isupper(),
            '-4:postag': postag1,
            '-4:word.cluster': cluster
        })
    else:
        features['Fourth_word'] = True
    
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:word.cluster': cluster
        })
    else:
        features['EOS'] = True
    if i < len(sent)-2 and (JANELA==2 or JANELA==3 or JANELA==4):
        word1 = sent[i+2][0]
        postag1 = sent[i+2][1]
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+2:word.lower()': word1.lower(),
            '+2:word.istitle()': word1.istitle(),
            '+2:word.isupper()': word1.isupper(),
            '+2:postag': postag1,
            '+2:word.cluster': cluster
        })
    else:
        features['Second_to_last'] = True
    if i < len(sent)-3 and (JANELA==3 or JANELA==4):
        word1 = sent[i+3][0]
        postag1 = sent[i+3][1]
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+3:word.lower()': word1.lower(),
            '+3:word.istitle()': word1.istitle(),
            '+3:word.isupper()': word1.isupper(),
            '+3:postag': postag1,
            '+3:word.cluster': cluster
        })
    else:
        features['Third_to_last'] = True
    
    if i < len(sent)-4 and JANELA==4:
        word1 = sent[i+4][0]
        postag1 = sent[i+4][1]
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+4:word.lower()': word1.lower(),
            '+4:word.istitle()': word1.istitle(),
            '+4:word.isupper()': word1.isupper(),
            '+4:postag': postag1,
            '+4:word.cluster': cluster
        })
    else:
        features['Fourth_to_last'] = True
    

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

#word2cluster = read_clusters(r"cluster/cluster-"+str(CLUSTER)+".tsv")

import joblib
import os

#AVAL no conjunto de teste do ner (flat)
print('NER flat, clustr 50, janela 3')


def getTiposEntidade():
    return ['Problema','Teste','Tratamento','Anatomia']

JANELA=4
CLUSTER=300

word2cluster = read_clusters(r"cluster/cluster-"+str(CLUSTER)+".tsv")
# BATCH 800

dicSentences_new_test = f.loadSentencesTest()
print(len(dicSentences_new_test))
dicSentences_new_test = {k: v for k, v in dicSentences_new_test.items() if k<BATCH}
print(dicSentences_new_test[0])
#print(dicSentences_new_test[27])
print('numero de sentencas no total:', len(dicSentences_new_test))

'''
example_sent = testdata[18]
print("\nSentence:", ' '.join(sent2tokens(example_sent)))
print(' '.join(sent2tokens(example_sent)))
print("Predicted:", ' '.join(crf.predict([sent2features(example_sent)])[0]))
print("Correct:  ", ' '.join(sent2labels(example_sent)))
'''

sentences=list()
frases=list()
allTokens=list()
tokens=list()
sentences=list()
frases=list()
allTokens=list()
listatokens=list()
for key, value in dicSentences_new_test.items():
    if key<BATCH:
        tokens = value[0]
        for token in tokens:
            listatokens.append(token[0])
            if len(token)>0:
                frases.append(tuple(token))
        sentences.append(frases)
        allTokens.append(listatokens)
        listatokens=list()
        frases=list()
print('sentences[0]:', sentences[0])
print(allTokens[0])

#testdata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]

def getTiposEntidade():
    return ['Problema','Teste','Tratamento','Anatomia']

#X_test = [sent2tokensPredict(s) for s in sentences]
X_test = [sent2features(s) for s in sentences]
#print('X_test[0]:', X_test[0])
print('len(sentences):', len(sentences))
print('len(allTokens):', len(allTokens))

OUTPUT_PATH = "CRF"
OUTPUT_FILE = "crf_model_primeiro_nivel"
crf = joblib.load(os.path.join(OUTPUT_PATH, OUTPUT_FILE))
y_pred = crf.predict(X_test)
print('len(y_pred):', len(y_pred))
print('y_pred[1]:', y_pred[1])
print('allTokens[1]:', allTokens[1])

#for tags, tokens in zip(y_pred, allTokens):
dic_predictions = f.getDicPredictions(y_pred, allTokens)
region_true_list, region_pred_list, lista_erros = f.getListaRegionsTruePred(BATCH, dicSentences_new_test, dic_predictions)
print('-----Avaliando só modelo de NER com CRF, nested:-----')
print(classification_report(region_true_list, region_pred_list, digits=6))
print(confusion_matrix(region_true_list, region_pred_list))

#tags, tokens = f.predictBERTNER_IO(sentences, 'all')
#dic_predictions = f.getDicPredictions(tags, tokens)


NER flat, clustr 50, janela 3
Pegando sentencas de teste gabarito: dic_sentencesTest.pkl
506
[[['Lucas', 0, 43], [',', 1, 48], ['74', 2, 50], ['anos', 3, 53], ['.', 4, 57]], []]
numero de sentencas no total: 506
sentences[0]: [('Lucas', 0, 43), (',', 1, 48), ('74', 2, 50), ('anos', 3, 53), ('.', 4, 57)]
['Lucas', ',', '74', 'anos', '.']
len(sentences): 506
len(allTokens): 506
len(y_pred): 506
y_pred[1]: ['O', 'Problema', 'Problema', 'Problema', 'O', 'O', 'O', 'O', 'Problema', 'O', 'O', 'O', 'Tratamento', 'Tratamento', 'O', 'O', 'O', 'O', 'O']
allTokens[1]: ['Em', 'acompanhamento', 'no', 'ambualtorio', 'há', '5', 'anos', 'por', 'FA', ',', 'uso', 'de', 'marevan', '5mg', '1', 'x', 'ao', 'dia', '.']
-----Avaliando só modelo de NER com CRF, nested:-----
              precision    recall  f1-score   support

    Anatomia   0.690476  0.295918  0.414286       196
           O   0.000000  0.000000  0.000000       659
    Problema   0.359350  0.653846  0.463799       338
       Teste   0.512195 

### AVAL spans / regioes

In [14]:
#Só os de dentro??

In [15]:
dic_sentencesTest = dicSentences_new_test.copy()
dic_sentencesTest[2]

[[['Comorbidades', 0, 142],
  [':', 1, 154],
  ['DM', 2, 156],
  ['há', 3, 159],
  ['10', 4, 162],
  ['anos', 5, 165],
  ['em', 6, 170],
  ['uso', 7, 173],
  ['de', 8, 177],
  ['metformina', 9, 180],
  ['850mg', 10, 191],
  ['3', 11, 197],
  ['cp', 12, 199],
  ['/', 13, 201],
  ['dia', 14, 202],
  [',', 15, 205],
  ['acarbose', 16, 207],
  ['1', 17, 216],
  ['cp', 18, 218],
  ['/', 19, 220],
  ['dia', 20, 221],
  ['e', 21, 225],
  ['glicazida', 22, 227],
  ['60mg', 23, 237],
  ['2', 24, 242],
  ['cp', 25, 244],
  ['/', 26, 246],
  ['dia', 27, 247],
  ['e', 28, 251],
  ['insulina', 29, 253],
  ['(', 30, 262],
  ['24', 31, 263],
  ['-', 32, 266],
  ['0', 33, 268],
  ['-', 34, 270],
  ['24', 35, 272],
  [')', 36, 274],
  ['.', 37, 275]],
 [['Comorbidades', [0], 'Problema'],
  ['DM', [2], 'Problema'],
  ['metformina 850mg', [9, 10], 'Tratamento'],
  ['acarbose', [16], 'Tratamento'],
  ['glicazida 60mg', [22, 23], 'Tratamento'],
  ['insulina', [29], 'Tratamento']]]

In [16]:
NUM_JANELA=4

#def getListaPositiva(dic_sentences):
#dic_sentences = dic_sentencesTest
def getListaCombinacoesOld(dic_sentences):
    listaSentencas=list()
    lista=list()
    numJanela=NUM_JANELA
    for key, value in dic_sentences.items():
        #print('value:', value)
        entidades = value[1]
        if len(entidades)==0:
            continue
        listaIndicesEntidadesFrase=list()
        for entidade in entidades:
          listaIndicesEntidadesFrase.append(entidade[1])
        dicTokens={}
        for token in value[0]:
          #print('token:', token)
          dicTokens[token[1]]=token[0]
        for entidade in entidades:
            #print('entidade:', entidade)
            label = entidade[2]
            indiceEntidade1=entidade[1][0]
            indiceEntidade2=entidade[1][-1]
            vizinhosAntes = list()
            vizinhosDepois = list()
            #print('indiceEntidade:', indiceEntidade)
            for tokens in value[0]:
                indice=tokens[1]
                #print('token: {}, indice: {}'.format(tokens[0], indice))
                if indice+4>=indiceEntidade1 and indice<indiceEntidade1:
                    vizinhosAntes.append(tokens[0])
                if indice-4<=indiceEntidade2 and indice>indiceEntidade2:
                    vizinhosDepois.append(tokens[0])
            lista.append([entidade[0], vizinhosAntes, vizinhosDepois, label])
            # agora, os negativos, para entidades com mais de um token
            if len(entidade[0].split())>1:
              #print('entidade[0] mais de um token: {}'.format(entidade[0].split()))
              #listaCombinacoes = powerset(entidade[0].split())
              listaCombinacoes = powerset(entidade[1])
              #print(listaCombinacoes)
              # ver se alguma dessas é positiva
              for combinacao in listaCombinacoes:
                #print('combinacao:', combinacao)
                if combinacao in listaIndicesEntidadesFrase:
                  #print('teeeem:', combinacao)
                  pass
                else:
                  combinacaoTokens = [dicTokens[t] for t in combinacao]
                  vizinhosAntes = list()
                  vizinhosDepois = list()
                  indiceEntidade1=combinacao[0]
                  indiceEntidade2=combinacao[-1]
                  for tokens in value[0]:
                    indice=tokens[1]
                    #print('token: {}, indice: {}'.format(tokens[0], indice))
                    if indice+4>=indiceEntidade1 and indice<indiceEntidade1:
                        vizinhosAntes.append(tokens[0])
                    if indice-4<=indiceEntidade2 and indice>indiceEntidade2:
                        vizinhosDepois.append(tokens[0])
                  lista.append([' '.join(combinacaoTokens), vizinhosAntes, vizinhosDepois, 'O'])
        listaSentencas.append(lista)
        lista=list()
        #break
    return listaSentencas
listacombinacoes = getListaCombinacoesOld(dic_sentencesTest)
print(listacombinacoes[:2])

[[['FA', ['há', '5', 'anos', 'por'], [',', 'uso', 'de', 'marevan'], 'Problema'], ['marevan 5mg', ['FA', ',', 'uso', 'de'], ['1', 'x', 'ao', 'dia'], 'Tratamento'], ['marevan', ['FA', ',', 'uso', 'de'], ['5mg', '1', 'x', 'ao'], 'O'], ['5mg', [',', 'uso', 'de', 'marevan'], ['1', 'x', 'ao', 'dia'], 'O']], [['Comorbidades', [], [':', 'DM', 'há', '10'], 'Problema'], ['DM', ['Comorbidades', ':'], ['há', '10', 'anos', 'em'], 'Problema'], ['metformina 850mg', ['anos', 'em', 'uso', 'de'], ['3', 'cp', '/', 'dia'], 'Tratamento'], ['metformina', ['anos', 'em', 'uso', 'de'], ['850mg', '3', 'cp', '/'], 'O'], ['850mg', ['em', 'uso', 'de', 'metformina'], ['3', 'cp', '/', 'dia'], 'O'], ['acarbose', ['cp', '/', 'dia', ','], ['1', 'cp', '/', 'dia'], 'Tratamento'], ['glicazida 60mg', ['cp', '/', 'dia', 'e'], ['2', 'cp', '/', 'dia'], 'Tratamento'], ['glicazida', ['cp', '/', 'dia', 'e'], ['60mg', '2', 'cp', '/'], 'O'], ['60mg', ['/', 'dia', 'e', 'glicazida'], ['2', 'cp', '/', 'dia'], 'O'], ['insulina', ['cp'

In [17]:
dic_predictions_ner_bert[2]

[[['Comorbidades', 0],
  [':', 1],
  ['DM', 2],
  ['há', 3],
  ['10', 4],
  ['anos', 5],
  ['em', 6],
  ['uso', 7],
  ['de', 8],
  ['metformina', 9],
  ['850mg', 10],
  ['3', 11],
  ['cp', 12],
  ['/', 13],
  ['dia', 14],
  [',', 15],
  ['acarbose', 16],
  ['1', 17],
  ['cp', 18],
  ['/', 19],
  ['dia', 20],
  ['e', 21],
  ['glicazida', 22],
  ['60mg', 23],
  ['2', 24],
  ['cp', 25],
  ['/', 26],
  ['dia', 27],
  ['e', 28],
  ['insulina', 29],
  ['(', 30],
  ['24', 31],
  ['-', 32],
  ['0', 33],
  ['-', 34],
  ['24', 35],
  [')', 36],
  ['.', 37]],
 [['Comorbidades', [0], 'Problema'],
  ['DM', [2], 'Problema'],
  ['metformina 850mg', [9, 10], 'Tratamento'],
  ['acarbose', [16], 'Tratamento'],
  ['glicazida 60mg', [22, 23], 'Tratamento'],
  ['insulina', [29], 'Tratamento']]]

In [18]:
#def getListaPositiva(dic_sentences):
#dic_sentences = dic_sentencesTest
def getListaCombinacoes(dic_sentences):
    listaSentencas=list()
    lista=list()
    dicPred={}
    #numJanela=NUM_JANELA
    numJanela=4
    num1=-1
    num2=0
    for key, value in dic_sentences.items():
        num1=num1+1
        #print('value:', value)
        entidades = value[1]
        if len(entidades)==0:
            dicPred[num2]=num1
            continue
        dicPred[num2]=num1
        num2=num2+1
        listaIndicesEntidadesFrase=list()
        for entidade in entidades:
          listaIndicesEntidadesFrase.append(entidade[1])
        dicTokens={}
        for token in value[0]:
          #print('token:', token)
          dicTokens[token[1]]=token[0]
        for entidade in entidades:
            #print('entidade:', entidade)
            #label = entidade[2]
            label = 'ENT'
            indiceEntidade1=entidade[1][0]
            indiceEntidade2=entidade[1][-1]
            vizinhosAntes = list()
            vizinhosDepois = list()
            #print('indiceEntidade:', indiceEntidade)
            for tokens in value[0]:
                indice=tokens[1]
                #print('token: {}, indice: {}'.format(tokens[0], indice))
                if indice+numJanela>=indiceEntidade1 and indice<indiceEntidade1:
                    vizinhosAntes.append(tokens[0])
                if indice-numJanela<=indiceEntidade2 and indice>indiceEntidade2:
                    vizinhosDepois.append(tokens[0])
            lista.append([entidade[0], vizinhosAntes, vizinhosDepois, label])
            # agora, os negativos, para entidades com mais de um token
            if len(entidade[0].split())>1:
              #print('entidade[0] mais de um token: {}'.format(entidade[0].split()))
              #listaCombinacoes = powerset(entidade[0].split())
              listaCombinacoes = powerset(entidade[1])
              #print(listaCombinacoes)
              # ver se alguma dessas é positiva
              for combinacao in listaCombinacoes:
                #print('combinacao:', combinacao)
                if combinacao in listaIndicesEntidadesFrase:
                  #print('teeeem:', combinacao)
                  pass
                else:
                  combinacaoTokens = [dicTokens[t] for t in combinacao]
                  vizinhosAntes = list()
                  vizinhosDepois = list()
                  indiceEntidade1=combinacao[0]
                  indiceEntidade2=combinacao[-1]
                  for tokens in value[0]:
                    indice=tokens[1]
                    #print('token: {}, indice: {}'.format(tokens[0], indice))
                    if indice+numJanela>=indiceEntidade1 and indice<indiceEntidade1:
                        vizinhosAntes.append(tokens[0])
                    if indice-numJanela<=indiceEntidade2 and indice>indiceEntidade2:
                        vizinhosDepois.append(tokens[0])
                  lista.append([' '.join(combinacaoTokens), vizinhosAntes, vizinhosDepois, 'O'])
        listaSentencas.append(lista)
        lista=list()
        #break
    return listaSentencas, dicPred


In [19]:
listacombinacoesTest, dicPred = getListaCombinacoes(dic_sentencesTest)
listacombinacoesTest[0:2]

[[['FA', ['há', '5', 'anos', 'por'], [',', 'uso', 'de', 'marevan'], 'ENT'],
  ['marevan 5mg', ['FA', ',', 'uso', 'de'], ['1', 'x', 'ao', 'dia'], 'ENT'],
  ['marevan', ['FA', ',', 'uso', 'de'], ['5mg', '1', 'x', 'ao'], 'O'],
  ['5mg', [',', 'uso', 'de', 'marevan'], ['1', 'x', 'ao', 'dia'], 'O']],
 [['Comorbidades', [], [':', 'DM', 'há', '10'], 'ENT'],
  ['DM', ['Comorbidades', ':'], ['há', '10', 'anos', 'em'], 'ENT'],
  ['metformina 850mg',
   ['anos', 'em', 'uso', 'de'],
   ['3', 'cp', '/', 'dia'],
   'ENT'],
  ['metformina', ['anos', 'em', 'uso', 'de'], ['850mg', '3', 'cp', '/'], 'O'],
  ['850mg', ['em', 'uso', 'de', 'metformina'], ['3', 'cp', '/', 'dia'], 'O'],
  ['acarbose', ['cp', '/', 'dia', ','], ['1', 'cp', '/', 'dia'], 'ENT'],
  ['glicazida 60mg', ['cp', '/', 'dia', 'e'], ['2', 'cp', '/', 'dia'], 'ENT'],
  ['glicazida', ['cp', '/', 'dia', 'e'], ['60mg', '2', 'cp', '/'], 'O'],
  ['60mg', ['/', 'dia', 'e', 'glicazida'], ['2', 'cp', '/', 'dia'], 'O'],
  ['insulina', ['cp', '/', 'd

In [20]:
listacombinacoesTest

[[['FA', ['há', '5', 'anos', 'por'], [',', 'uso', 'de', 'marevan'], 'ENT'],
  ['marevan 5mg', ['FA', ',', 'uso', 'de'], ['1', 'x', 'ao', 'dia'], 'ENT'],
  ['marevan', ['FA', ',', 'uso', 'de'], ['5mg', '1', 'x', 'ao'], 'O'],
  ['5mg', [',', 'uso', 'de', 'marevan'], ['1', 'x', 'ao', 'dia'], 'O']],
 [['Comorbidades', [], [':', 'DM', 'há', '10'], 'ENT'],
  ['DM', ['Comorbidades', ':'], ['há', '10', 'anos', 'em'], 'ENT'],
  ['metformina 850mg',
   ['anos', 'em', 'uso', 'de'],
   ['3', 'cp', '/', 'dia'],
   'ENT'],
  ['metformina', ['anos', 'em', 'uso', 'de'], ['850mg', '3', 'cp', '/'], 'O'],
  ['850mg', ['em', 'uso', 'de', 'metformina'], ['3', 'cp', '/', 'dia'], 'O'],
  ['acarbose', ['cp', '/', 'dia', ','], ['1', 'cp', '/', 'dia'], 'ENT'],
  ['glicazida 60mg', ['cp', '/', 'dia', 'e'], ['2', 'cp', '/', 'dia'], 'ENT'],
  ['glicazida', ['cp', '/', 'dia', 'e'], ['60mg', '2', 'cp', '/'], 'O'],
  ['60mg', ['/', 'dia', 'e', 'glicazida'], ['2', 'cp', '/', 'dia'], 'O'],
  ['insulina', ['cp', '/', 'd

In [98]:
def getTiposEntidadeBinario():
    return ['O','ENT']

def getTiposEntidade():
    return ['Problema','Teste','Tratamento','Anatomia']
    
def read_clusters(cluster_file):
    word2cluster = {}
    try:
        with open(cluster_file, encoding='utf-8') as i:
            for num, line in enumerate(i):
                if line:
                    word, cluster = line.strip().split('\t')
                    word2cluster[word] = cluster
    except:
        print(line)
        print(num)
        raise
    return word2cluster


def word2features(sent, word2cluster, dicPostagger):
    try:
        features = list()
        entidades = sent[0]
        for entidade in entidades.split():
            postag = tipoPostaggerTokens(entidade, dicPostagger)
            features.extend([
            'bias',
            'word.lower=' + entidade.lower(),
            'word[-3:]=' + entidade[-3:],
            'word[:3]=' + entidade[:3],
            'word.isupper=%s' % entidade.isupper(),
            'word.istitle=%s' % entidade.istitle(),
            'word.isdigit=%s' % entidade.isdigit(),
            'word.cluster=%s' % word2cluster[entidade.lower()] if entidade.lower() in word2cluster else "0",
            'postag=' + postag
            ])
        # palavras anteriores
        vizinhosAntes = sent[1] 
        if len(vizinhosAntes)>0:
            for num, vizinhoAntes in enumerate(vizinhosAntes):
                word1 = vizinhoAntes
                postag1 =  tipoPostaggerTokens(vizinhoAntes, dicPostagger)
                features.extend([
                    '-'+str(num+1)+':word.lower=' + word1.lower(),
                    '-'+str(num+1)+':word.istitle=%s' % word1.istitle(),
                    '-'+str(num+1)+':word.isupper=%s' % word1.isupper(),
                    '-'+str(num+1)+':postag=' + postag1
                ])
        else:
            features.append('BOS')

        # próximas palavras
        vizinhosDepois = sent[2]
        if len(vizinhosDepois)>0:
            for num, vizinhoDepois in enumerate(vizinhosDepois):
                word1 = vizinhoDepois
                postag1 =  tipoPostaggerTokens(vizinhoDepois, dicPostagger)
                features.extend([
                    '+'+str(num+1)+':word.lower=' + word1.lower(),
                    '+'+str(num+1)+':word.istitle=%s' % word1.istitle(),
                    '+'+str(num+1)+':word.isupper=%s' % word1.isupper(),
                    '+'+str(num+1)+':postag=' + postag1
                ])
        else:
            features.append('EOS')
    except:
        print('senttt:', sent)
        raise
    return features


def sent2features(lista, word2cluster, dicPostagger):
    #return word2features(lista, word2cluster, dicPostagger)
    return [word2features(lista[i], word2cluster, dicPostagger) for i in range(len(lista))]

def sent2labels(lista):
    #return [label for _, _, _, label in lista]
    return [label for _, _, _, label in lista]

#def sent2tokens(sent):
#    return [token for token, postag, label in sent]

#word2cluster = read_clusters(r"cluster/cluster-50.tsv")
#word2cluster = read_clusters(r"clusters/cluster-5.tsv")
#word2cluster = read_clusters(r"clusters/cluster-10.tsv")
#word2cluster = read_clusters(r"clusters/cluster-100.tsv")
#word2cluster = read_clusters(r"clusters/cluster-300.tsv")

In [22]:
import joblib
import os

CLUSTER=300
word2cluster = read_clusters(r"cluster/cluster-300.tsv")

X_test = [sent2features(s, word2cluster, dicPostagger) for s in listacombinacoesTest]
y_test = [sent2labels(s) for s in listacombinacoesTest]

print('regiao CRF binario, cluster 300, janela 4 => sem estar no pipeline')

OUTPUT_PATH = "CRF"
OUTPUT_FILE = "crf_model_binario_regiao"
crf = joblib.load(os.path.join(OUTPUT_PATH, OUTPUT_FILE))
y_pred = crf.predict(X_test)
print(flat_classification_report(
    y_test, y_pred, labels=getTiposEntidadeBinario(), digits=3
))

regiao CRF binario, cluster 300, janela 4 => sem estar no pipeline
              precision    recall  f1-score   support

           O      0.948     0.944     0.946      2730
         ENT      0.848     0.859     0.853       991

    accuracy                          0.921      3721
   macro avg      0.898     0.901     0.900      3721
weighted avg      0.922     0.921     0.921      3721



C:\Users\lisat\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=['O', 'ENT'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [23]:
dic_predictions_ner_bert[15]

[[['calcificação', 0],
  ['mitral', 1],
  ['e', 2],
  ['aórtica', 3],
  ['com', 4],
  ['refluxo', 5],
  ['leve', 6],
  ['.', 7]],
 [['calcificação', [0], 'Problema'],
  ['mitral', [1], 'Tratamento'],
  ['e', [2], 'Problema'],
  ['aórtica', [3], 'Anatomia'],
  ['refluxo leve', [5, 6], 'Problema']]]

In [128]:
CLUSTER=300
word2cluster = read_clusters(r"cluster/cluster-300.tsv")
#dic_predictions = f.getDicPredictions(y_pred, allTokens)

print('agora, com pipeline... regiao CRF binario, cluster 300, janela 4 => com pipeline')

#listacombinacoesTestPred, dicPred = getListaCombinacoes(dic_predictions)
listacombinacoesTestPred, dicPred = getListaCombinacoes(dic_predictions_ner_bert)

X_test = [sent2features(s, word2cluster, dicPostagger) for s in listacombinacoesTestPred]
#y_test = [sent2labels(s) for s in listacombinacoesTest]

OUTPUT_PATH = "CRF"
OUTPUT_FILE = "crf_model_binario_regiao"
crf = joblib.load(os.path.join(OUTPUT_PATH, OUTPUT_FILE))
y_pred = crf.predict(X_test)

dic_sentencesTest2={}
num=0
for key, value in dic_sentencesTest.items():
    if value[1]:
        dic_sentencesTest2[num] = dic_sentencesTest[key]
        num=num+1
        
        #dic_sentencesTest
    
regiao_true=list()
regiao_pred=list()

dicLabels={'O':'0', 'ENT':'1', 'Problema':'1', 'Tratamento':'1', 'Teste':'1', 'Anatomia':'1'}

num=0
for ent, label in zip(listacombinacoesTestPred, y_pred):
    #print(num)
    #print(ent)
    numKey=dicPred[num]
    listaTrue=dic_sentencesTest[numKey][1]
    num=num+1
    #print(listaTrue)
    for entidade in ent:
        textoEntidade=entidade[0]
        labelEntidade=dicLabels[entidade[3]]
        labelEntidadeTrue='0'
        #print(textoEntidade)
        tem=0
        for entTrue in listaTrue:
            textoEntidadeTrue=entTrue[0]
            if textoEntidade == textoEntidadeTrue:
                #print('temmm:', textoEntidade)
                labelEntidadeTrue = dicLabels[entTrue[2]]
                tem=1
                break
        if tem==1:
            if labelEntidade == labelEntidadeTrue:
                regiao_true.append(labelEntidadeTrue)
                regiao_pred.append(labelEntidade)
            else:
                #print('aaaaaaaaaaaaa',textoEntidade)
                #print(num)
                regiao_true.append(labelEntidadeTrue)
                regiao_pred.append(labelEntidade)
        else:
            #if labelEntidade!='0':
                #print('nao temmm:{}, {}'.format(textoEntidade,labelEntidade))
                #print(num)
            regiao_true.append(labelEntidadeTrue)
            regiao_pred.append(labelEntidade)
            
        
                
    #if num>40:
    #    break
print('len(regiao_pred):', len(regiao_pred))
print('len(regiao_true):', len(regiao_true))

print('regiao_pred[:4]:', regiao_pred[40:74])
print('regiao_true[:4]:', regiao_true[40:74])

print(flat_classification_report(
    regiao_pred, regiao_true, labels=['1','0'], digits=3
))

agora, com pipeline... regiao CRF binario, cluster 300, janela 4 => com pipeline
len(regiao_pred): 3355
len(regiao_true): 3355
regiao_pred[:4]: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1']
regiao_true[:4]: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1']
              precision    recall  f1-score   support

           1      0.841     0.849     0.845       906
           0      0.944     0.941     0.942      2449

    accuracy                          0.916      3355
   macro avg      0.893     0.895     0.894      3355
weighted avg      0.916     0.916     0.916      3355



In [25]:
#def getListaPositiva(dic_sentences):
#dic_sentences = dic_sentencesTest
def getListaCombinacoesAll(dic_sentences):
    listaSentencas=list()
    lista=list()
    dicPred={}
    #numJanela=NUM_JANELA
    numJanela=4
    num1=-1
    num2=0
    for key, value in dic_sentences.items():
        num1=num1+1
        #print('value:', value)
        entidades = value[1]
        if len(entidades)==0:
            dicPred[num2]=num1
            continue
        dicPred[num2]=num1
        num2=num2+1
        listaIndicesEntidadesFrase=list()
        for entidade in entidades:
          listaIndicesEntidadesFrase.append(entidade[1])
        dicTokens={}
        for token in value[0]:
          #print('token:', token)
          dicTokens[token[1]]=token[0]
        for entidade in entidades:
            #print('entidade:', entidade)
            label = entidade[2]
            #label = 'ENT'
            indiceEntidade1=entidade[1][0]
            indiceEntidade2=entidade[1][-1]
            vizinhosAntes = list()
            vizinhosDepois = list()
            #print('indiceEntidade:', indiceEntidade)
            for tokens in value[0]:
                indice=tokens[1]
                #print('token: {}, indice: {}'.format(tokens[0], indice))
                if indice+numJanela>=indiceEntidade1 and indice<indiceEntidade1:
                    vizinhosAntes.append(tokens[0])
                if indice-numJanela<=indiceEntidade2 and indice>indiceEntidade2:
                    vizinhosDepois.append(tokens[0])
            lista.append([entidade[0], vizinhosAntes, vizinhosDepois, label])
            # agora, os negativos, para entidades com mais de um token
            if len(entidade[0].split())>1:
              #print('entidade[0] mais de um token: {}'.format(entidade[0].split()))
              #listaCombinacoes = powerset(entidade[0].split())
              listaCombinacoes = powerset(entidade[1])
              #print(listaCombinacoes)
              # ver se alguma dessas é positiva
              for combinacao in listaCombinacoes:
                #print('combinacao:', combinacao)
                if combinacao in listaIndicesEntidadesFrase:
                  #print('teeeem:', combinacao)
                  pass
                else:
                  combinacaoTokens = [dicTokens[t] for t in combinacao]
                  vizinhosAntes = list()
                  vizinhosDepois = list()
                  indiceEntidade1=combinacao[0]
                  indiceEntidade2=combinacao[-1]
                  for tokens in value[0]:
                    indice=tokens[1]
                    #print('token: {}, indice: {}'.format(tokens[0], indice))
                    if indice+numJanela>=indiceEntidade1 and indice<indiceEntidade1:
                        vizinhosAntes.append(tokens[0])
                    if indice-numJanela<=indiceEntidade2 and indice>indiceEntidade2:
                        vizinhosDepois.append(tokens[0])
                  lista.append([' '.join(combinacaoTokens), vizinhosAntes, vizinhosDepois, 'O'])
        listaSentencas.append(lista)
        lista=list()
        #break
    return listaSentencas, dicPred

listacombinacoesTestPred, dicPred = getListaCombinacoesAll(dic_predictions_ner_bert)
listacombinacoesTestPred[:2]


[[['FA',
   ['há', '5', 'anos', 'por'],
   [',', 'uso', 'de', 'marevan'],
   'Problema'],
  ['marevan 5mg',
   ['FA', ',', 'uso', 'de'],
   ['1', 'x', 'ao', 'dia'],
   'Tratamento'],
  ['marevan', ['FA', ',', 'uso', 'de'], ['5mg', '1', 'x', 'ao'], 'O'],
  ['5mg', [',', 'uso', 'de', 'marevan'], ['1', 'x', 'ao', 'dia'], 'O']],
 [['Comorbidades', [], [':', 'DM', 'há', '10'], 'Problema'],
  ['DM', ['Comorbidades', ':'], ['há', '10', 'anos', 'em'], 'Problema'],
  ['metformina 850mg',
   ['anos', 'em', 'uso', 'de'],
   ['3', 'cp', '/', 'dia'],
   'Tratamento'],
  ['metformina', ['anos', 'em', 'uso', 'de'], ['850mg', '3', 'cp', '/'], 'O'],
  ['850mg', ['em', 'uso', 'de', 'metformina'], ['3', 'cp', '/', 'dia'], 'O'],
  ['acarbose', ['cp', '/', 'dia', ','], ['1', 'cp', '/', 'dia'], 'Tratamento'],
  ['glicazida 60mg',
   ['cp', '/', 'dia', 'e'],
   ['2', 'cp', '/', 'dia'],
   'Tratamento'],
  ['glicazida', ['cp', '/', 'dia', 'e'], ['60mg', '2', 'cp', '/'], 'O'],
  ['60mg', ['/', 'dia', 'e', 'gli

In [26]:
def read_clusters(cluster_file):
    word2cluster = {}
    try:
        with open(cluster_file, encoding='utf-8') as i:
            for num, line in enumerate(i):
                if line:
                    word, cluster = line.strip().split('\t')
                    word2cluster[word] = cluster
    except:
        print(line)
        print(num)
        raise
    return word2cluster


def word2features(sent, word2cluster, dicPostagger):
    try:
        features = list()
        entidades = sent[0]
        for entidade in entidades.split():
            postag = tipoPostaggerTokens(entidade, dicPostagger)
            features.extend([
            'bias',
            'word.lower=' + entidade.lower(),
            'word[-3:]=' + entidade[-3:],
            'word[-2:]=' + entidade[-2:],
            'word.isupper=%s' % entidade.isupper(),
            'word.istitle=%s' % entidade.istitle(),
            'word.isdigit=%s' % entidade.isdigit(),
            'word.cluster=%s' % word2cluster[entidade.lower()] if entidade.lower() in word2cluster else "0",
            'postag=' + postag
            ])
        # palavras anteriores
        vizinhosAntes = sent[1] 
        if len(vizinhosAntes)>0:
            for num, vizinhoAntes in enumerate(vizinhosAntes):
                word1 = vizinhoAntes
                postag1 =  tipoPostaggerTokens(vizinhoAntes, dicPostagger)
                features.extend([
                    '-'+str(num+1)+':word.lower=' + word1.lower(),
                    '-'+str(num+1)+':word.istitle=%s' % word1.istitle(),
                    '-'+str(num+1)+':word.isupper=%s' % word1.isupper(),
                    '-'+str(num+1)+':postag=' + postag1
                ])
        else:
            features.append('BOS')

        # próximas palavras
        vizinhosDepois = sent[2]
        if len(vizinhosDepois)>0:
            for num, vizinhoDepois in enumerate(vizinhosDepois):
                word1 = vizinhoDepois
                postag1 =  tipoPostaggerTokens(vizinhoDepois, dicPostagger)
                features.extend([
                    '+'+str(num+1)+':word.lower=' + word1.lower(),
                    '+'+str(num+1)+':word.istitle=%s' % word1.istitle(),
                    '+'+str(num+1)+':word.isupper=%s' % word1.isupper(),
                    '+'+str(num+1)+':postag=' + postag1
                ])
        else:
            features.append('EOS')
    except:
        print('sent:', sent)
        raise
    return features


def sent2features(lista, word2cluster, dicPostagger):
    #return word2features(lista, word2cluster, dicPostagger)
    return [word2features(lista[i], word2cluster, dicPostagger) for i in range(len(lista))]

def sent2labels(lista):
    #return [label for _, _, _, label in lista]
    return [label for _, _, _, label in lista]


def sent2tokens(sent):
    return [token for token, postag, label in sent]

word2cluster = read_clusters(r"cluster/cluster-50.tsv")
#word2cluster = read_clusters(r"clusters/cluster-5.tsv")
#word2cluster = read_clusters(r"clusters/cluster-10.tsv")
#word2cluster = read_clusters(r"clusters/cluster-100.tsv")
#word2cluster = read_clusters(r"clusters/cluster-300.tsv")

#dic_predictions = f.getDicPredictions(y_pred, allTokens)

print('agora, com pipeline... regiao CRF binario, cluster 300, janela 4 => com pipeline')

#listacombinacoesTestPred, dicPred = getListaCombinacoes(dic_predictions)
listacombinacoesTestPred, dicPred = getListaCombinacoesAll(dic_predictions_ner_bert)

X_test = [sent2features(s, word2cluster, dicPostagger) for s in listacombinacoesTestPred]
#y_test = [sent2labels(s) for s in listacombinacoesTest]

OUTPUT_PATH = "CRF"
OUTPUT_FILE = "crf_model_regiao_cluster50"
crf = joblib.load(os.path.join(OUTPUT_PATH, OUTPUT_FILE))
y_pred = crf.predict(X_test)

dic_sentencesTest2={}
num=0
for key, value in dic_sentencesTest.items():
    if value[1]:
        dic_sentencesTest2[num] = dic_sentencesTest[key]
        num=num+1
        
        #dic_sentencesTest
    
regiao_true=list()
regiao_pred=list()

dicLabels={'O':'0', 'ENT':'1', 'Problema':'1', 'Tratamento':'2', 'Teste':'3', 'Anatomia':'4'}

def getTiposEntidade():
    return ['O','Problema','Teste','Tratamento','Anatomia']

def getTiposEntidadeInt():
    return [0,1,2,3,4]

def getTiposEntidadeIntSemO():
    return [1,2,3,4]

num=0
for ent, label in zip(listacombinacoesTestPred, y_pred):
    #print(num)
    #print(ent)
    numKey=dicPred[num]
    listaTrue=dic_sentencesTest[numKey][1]
    num=num+1
    #print(listaTrue)
    for entidade in ent:
        textoEntidade=entidade[0]
        labelEntidade=dicLabels[entidade[3]]
        labelEntidadeTrue='0'
        #print(textoEntidade)
        tem=0
        for entTrue in listaTrue:
            textoEntidadeTrue=entTrue[0]
            if textoEntidade == textoEntidadeTrue:
                #print('temmm:', textoEntidade)
                labelEntidadeTrue = dicLabels[entTrue[2]]
                tem=1
                break
        if tem==1:
            if labelEntidade == labelEntidadeTrue:
                regiao_true.append(labelEntidadeTrue)
                regiao_pred.append(labelEntidade)
            else:
                #print('aaaaaaaaaaaaa',textoEntidade)
                #print(num)
                regiao_true.append(labelEntidadeTrue)
                regiao_pred.append(labelEntidade)
        else:
            #if labelEntidade!='0':
                #print('nao temmm:{}, {}'.format(textoEntidade,labelEntidade))
                #print(num)
            regiao_true.append(labelEntidadeTrue)
            regiao_pred.append(labelEntidade)
                
    #if num>40:
    #    break
print('len(regiao_pred):', len(regiao_pred))
print('len(regiao_true):', len(regiao_true))

print('regiao_pred[:4]:', regiao_pred[40:74])
print('regiao_true[:4]:', regiao_true[40:74])

print(flat_classification_report(
    regiao_pred, regiao_true, labels=getTiposEntidadeInt(), digits=3
))

print(flat_classification_report(
    regiao_pred, regiao_true, labels=getTiposEntidadeIntSemO(), digits=3
))

agora, com pipeline... regiao CRF binario, cluster 300, janela 4 => com pipeline
len(regiao_pred): 3355
len(regiao_true): 3355
regiao_pred[:4]: ['4', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '4', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1']
regiao_true[:4]: ['4', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '4', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1']
              precision    recall  f1-score   support

           0      0.944     0.941     0.942      2449
           1      0.956     0.798     0.870       357
           2      0.906     0.828     0.865       221
           3      0.956     0.886     0.920       246
           4      0.382     0.866     0.530        82

   micro avg      0.912     0.912     0.912      3355
   macro avg      0.829     0.864     0.825      3355
weighted avg      0.930     0.912     0.918      3355

        

C:\Users\lisat\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=[0, 1, 2, 3, 4] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\lisat\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:576: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)
C:\Users\lisat\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=[1, 2, 3, 4] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [27]:
dic_predictions[15]

[[['calcificação', 0],
  ['mitral', 1],
  ['e', 2],
  ['aórtica', 3],
  ['com', 4],
  ['refluxo', 5],
  ['leve', 6],
  ['.', 7]],
 [['calcificação mitral', [0, 1], 'Problema'],
  ['aórtica', [3], 'Teste'],
  ['refluxo leve', [5, 6], 'Problema']]]

In [28]:
listacombinacoesTestPred[:3]

[[['FA',
   ['há', '5', 'anos', 'por'],
   [',', 'uso', 'de', 'marevan'],
   'Problema'],
  ['marevan 5mg',
   ['FA', ',', 'uso', 'de'],
   ['1', 'x', 'ao', 'dia'],
   'Tratamento'],
  ['marevan', ['FA', ',', 'uso', 'de'], ['5mg', '1', 'x', 'ao'], 'O'],
  ['5mg', [',', 'uso', 'de', 'marevan'], ['1', 'x', 'ao', 'dia'], 'O']],
 [['Comorbidades', [], [':', 'DM', 'há', '10'], 'Problema'],
  ['DM', ['Comorbidades', ':'], ['há', '10', 'anos', 'em'], 'Problema'],
  ['metformina 850mg',
   ['anos', 'em', 'uso', 'de'],
   ['3', 'cp', '/', 'dia'],
   'Tratamento'],
  ['metformina', ['anos', 'em', 'uso', 'de'], ['850mg', '3', 'cp', '/'], 'O'],
  ['850mg', ['em', 'uso', 'de', 'metformina'], ['3', 'cp', '/', 'dia'], 'O'],
  ['acarbose', ['cp', '/', 'dia', ','], ['1', 'cp', '/', 'dia'], 'Tratamento'],
  ['glicazida 60mg',
   ['cp', '/', 'dia', 'e'],
   ['2', 'cp', '/', 'dia'],
   'Tratamento'],
  ['glicazida', ['cp', '/', 'dia', 'e'], ['60mg', '2', 'cp', '/'], 'O'],
  ['60mg', ['/', 'dia', 'e', 'gli

In [29]:
y_pred[13]

['O', 'O', 'O', 'O', 'O', 'O', 'O']

In [30]:
dicPred

{0: 1,
 1: 2,
 2: 3,
 3: 4,
 4: 6,
 5: 7,
 6: 8,
 7: 9,
 8: 10,
 9: 11,
 10: 12,
 11: 13,
 12: 14,
 13: 15,
 14: 16,
 15: 17,
 16: 18,
 17: 19,
 18: 20,
 19: 23,
 20: 24,
 21: 25,
 22: 26,
 23: 27,
 24: 28,
 25: 30,
 26: 31,
 27: 32,
 28: 33,
 29: 34,
 30: 35,
 31: 36,
 32: 37,
 33: 38,
 34: 41,
 35: 42,
 36: 43,
 37: 44,
 38: 45,
 39: 46,
 40: 47,
 41: 48,
 42: 49,
 43: 50,
 44: 51,
 45: 52,
 46: 53,
 47: 54,
 48: 56,
 49: 58,
 50: 59,
 51: 60,
 52: 61,
 53: 62,
 54: 63,
 55: 64,
 56: 65,
 57: 66,
 58: 67,
 59: 68,
 60: 69,
 61: 70,
 62: 71,
 63: 72,
 64: 74,
 65: 75,
 66: 76,
 67: 77,
 68: 78,
 69: 79,
 70: 80,
 71: 82,
 72: 83,
 73: 84,
 74: 85,
 75: 86,
 76: 87,
 77: 88,
 78: 89,
 79: 90,
 80: 93,
 81: 94,
 82: 95,
 83: 97,
 84: 98,
 85: 99,
 86: 101,
 87: 103,
 88: 104,
 89: 105,
 90: 106,
 91: 107,
 92: 109,
 93: 110,
 94: 111,
 95: 112,
 96: 113,
 97: 114,
 98: 115,
 99: 116,
 100: 117,
 101: 118,
 102: 119,
 103: 121,
 104: 123,
 105: 124,
 106: 130,
 107: 131,
 108: 132,
 109:

In [68]:
#formato:
#dicPred
listaPredictRegiao = list()
listaPredictAllRegiao = list()
listaPredictFrases = list()
listaPredictAllFrases = list()
#dicPred2={}
num=0
pred_region_labels = list()
pred_region_label = list()

for labels, regioes in zip(y_pred, listacombinacoesTestPred):
    #print('labels:', labels)
    #print('regioes:', regioes)
    #print('num:', num)
    numKey=dicPred[num]
    #listaTrue=dic_sentencesTest[numKey][1]
    allTokens = dic_sentencesTest[numKey][0]
    #allTokens = dic_sentencesTest[num][0]
    frase=' '.join([a[0] for a in allTokens])
    #print('frase:', frase)
    for label, regiao in zip(labels, regioes):
        #print(label)
        #print(regiao)
        #listaPredictRegiao.append([regiao, frase, indices, label])
        if label == 'ENT' or label == 'Problema' or label == 'Teste' or label == 'Anatomia' or label == 'Tratamento':
        #if label == 1 or label == 2 or label == 3 or label == 4:
            listaPredictRegiao.append([regiao[0], frase])
            pred_region_label.append(label)
        listaPredictAllRegiao.append([regiao[0], frase])
    pred_region_labels.append(pred_region_label)
    pred_region_label = list()
    #if len(listaPredictRegiao)>0:
    #    listaPredictFrases.append(listaPredictRegiao)
    #    listaPredictRegiao = list()
    listaPredictFrases.append(listaPredictRegiao)
    listaPredictAllFrases.append(listaPredictAllRegiao)
    listaPredictRegiao = list()
    listaPredictAllRegiao = list()
    #else:
    #    print('naooo tem num:', num)
    num=num+1
    #dicPred2[num] = numKey
    
print(listaPredictFrases[2])
#dicPred2


[['HAS', 'HAS há 15 anos em uso de losartana 50mg / dia e digoxina 1 / 2 cp / dia , carvedilol 25 12 / 12 , HCTZ .'], ['losartana 50mg', 'HAS há 15 anos em uso de losartana 50mg / dia e digoxina 1 / 2 cp / dia , carvedilol 25 12 / 12 , HCTZ .'], ['carvedilol 25', 'HAS há 15 anos em uso de losartana 50mg / dia e digoxina 1 / 2 cp / dia , carvedilol 25 12 / 12 , HCTZ .'], ['HCTZ', 'HAS há 15 anos em uso de losartana 50mg / dia e digoxina 1 / 2 cp / dia , carvedilol 25 12 / 12 , HCTZ .']]


In [69]:
listaPredictAllFrases

[[['FA',
   'Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .'],
  ['marevan 5mg',
   'Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .'],
  ['marevan',
   'Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .'],
  ['5mg',
   'Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .']],
 [['Comorbidades',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'],
  ['DM',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'],
  ['metformina 850mg',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'],
  ['metformina',
   'Comorbidades : DM há 10 anos em uso de metformina 850

In [78]:
'''dic_predictions_sentence = {}
num=-1
for labels, regiao in zip(y_pred, listaPredictAllFrases):
    num=num+1
    ents = []
    for label, ent in zip(labels, regiao):
        ents.append([label, ent[0]])
    valores = [dic_predictions[num][0], ents]
    ents=[]
    dic_predictions_sentence[num]=valores
    if num>4:
        break
print(dic_predictions_sentence)
'''

{0: [[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], [['Problema', 'FA'], ['Tratamento', 'marevan 5mg'], ['O', 'marevan'], ['O', '5mg']]], 1: [[['Em', 0], ['acompanhamento', 1], ['no', 2], ['ambualtorio', 3], ['há', 4], ['5', 5], ['anos', 6], ['por', 7], ['FA', 8], [',', 9], ['uso', 10], ['de', 11], ['marevan', 12], ['5mg', 13], ['1', 14], ['x', 15], ['ao', 16], ['dia', 17], ['.', 18]], [['Problema', 'Comorbidades'], ['Problema', 'DM'], ['Tratamento', 'metformina 850mg'], ['O', 'metformina'], ['O', '850mg'], ['O', 'acarbose'], ['Tratamento', 'glicazida 60mg'], ['O', 'glicazida'], ['O', '60mg'], ['Tratamento', 'insulina']]], 2: [[['Comorbidades', 0], [':', 1], ['DM', 2], ['há', 3], ['10', 4], ['anos', 5], ['em', 6], ['uso', 7], ['de', 8], ['metformina', 9], ['850mg', 10], ['3', 11], ['cp', 12], ['/', 13], ['dia', 14], [',', 15], ['acarbose', 16], ['1', 17], ['cp', 18], ['/', 19], ['dia', 20], ['e', 21], ['glicazida', 22], ['60mg', 23], ['2', 24], ['cp', 25], ['/', 26], ['dia

In [32]:
pred_region_labels

[['Problema', 'Tratamento'],
 ['Problema', 'Problema', 'Tratamento', 'Tratamento', 'Tratamento'],
 ['Problema', 'Tratamento', 'Tratamento', 'Tratamento'],
 ['Problema', 'Tratamento'],
 ['Problema', 'Problema', 'Problema', 'Problema', 'Problema'],
 ['Problema', 'Teste', 'Teste', 'Problema'],
 ['Problema'],
 ['Anatomia', 'Problema'],
 ['Anatomia', 'Problema'],
 ['Teste', 'Teste'],
 ['Teste', 'Teste'],
 ['Teste'],
 ['Problema'],
 [],
 ['Problema'],
 ['Teste', 'Teste', 'Problema'],
 ['Teste', 'Problema'],
 ['Problema', 'Anatomia'],
 ['Tratamento', 'Tratamento', 'Tratamento', 'Tratamento'],
 ['Problema',
  'Problema',
  'Problema',
  'Problema',
  'Tratamento',
  'Problema',
  'Teste',
  'Problema',
  'Problema',
  'Anatomia',
  'Anatomia',
  'Problema',
  'Anatomia',
  'Problema',
  'Problema',
  'Anatomia',
  'Problema'],
 ['Problema', 'Anatomia', 'Problema', 'Problema', 'Anatomia', 'Problema'],
 ['Problema', 'Problema'],
 ['Problema'],
 ['Problema'],
 ['Problema'],
 ['Problema', 'Anatomi

In [59]:
listaPredictFrases[:2]

[[['FA',
   'Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .'],
  ['marevan 5mg',
   'Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .']],
 [['Comorbidades',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'],
  ['DM',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'],
  ['metformina 850mg',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'],
  ['glicazida 60mg',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'],
  ['insulina',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / di

In [34]:
y_pred

[['Problema', 'Tratamento', 'O', 'O'],
 ['Problema',
  'Problema',
  'Tratamento',
  'O',
  'O',
  'O',
  'Tratamento',
  'O',
  'O',
  'Tratamento'],
 ['Problema',
  'Tratamento',
  'O',
  'O',
  'O',
  'Tratamento',
  'O',
  'O',
  'Tratamento'],
 ['Problema', 'Tratamento', 'O'],
 ['Problema', 'Problema', 'Problema', 'Problema', 'Problema'],
 ['Problema', 'Teste', 'Teste', 'Problema'],
 ['Problema'],
 ['Anatomia', 'Problema'],
 ['Anatomia', 'Problema', 'O'],
 ['Teste', 'Teste'],
 ['Teste', 'Teste', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['Teste',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['Problema',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['Problema'],
 ['Teste', 'Teste', 'Problema'],
 ['Teste', 'Problema'],
 ['Problema', 'O', 'O', 'O', 'O', 'Anatomia'],
 ['Tratamento',
  'O',
  'O',
  'O',
  '

In [122]:
regiao_true=list()
regiao_pred=list()

getLabels=['Problema', 'Tratamento', 'Teste', 'Anatomia']

#dicLabels={'O':'0', 'ENT':'1', 'Problema':'1', 'Tratamento':'1', 'Teste':'1', 'Anatomia':'1'}
#dicLabels={'O':0, 'ENT':1, 'Problema':1, 'Tratamento':1, 'Teste':1, 'Anatomia':1}
dicLabels={'Problema':'0', 'Tratamento':'1', 'Teste':'2', 'Anatomia':'3', 'O':'4'}
#dicLabels={0:'Problema', 1:'Tratamento', 2:'Teste', 3:'Anatomia'}
#labels = {0:'Problema', 1:'Tratamento', 2:'Teste', 3:'Anatomia'}

num=0
for ent, label in zip(listaPredictFrases, pred_region_labels):
    #print('--ent--', ent)
    #print('label:', label)
    numKey=dicPred[num]
    listaTrue=dic_sentencesTest[numKey][1]
    num=num+1
    if num<BATCH:    
        #print('num:', num)
        #print('listaTrue:', listaTrue)
        #print('ent:',ent)
        for numEnt, entidade in enumerate(ent):
            #print('entidade:', entidade)
            #print('numEnt:', numEnt)
            #print('label[numEnt]:', label[numEnt])
            textoEntidade=entidade[0]
            labelEntidade=dicLabels[label[numEnt]]
            #labelEntidade=dicLabels[label[numEnt]]
            #labelEntidade='0'
            #labelEntidadeTrue='Problema'
            labelEntidadeTrue='4'
            
            tem=0
            '''
            print('listaTrue[numEnt]:', listaTrue[numEnt])
            if listaTrue[numEnt][0] == textoEntidade:
                #tem=1
                labelEntidadeTrue = dicLabels[listaTrue[numEnt][2]]
            print('labelEntidadeTrue:', labelEntidadeTrue)
            print('labelEntidade:', labelEntidade)
            regiao_true.append(labelEntidadeTrue)
            regiao_pred.append(labelEntidade)
            '''

            for entTrue in listaTrue:
                textoEntidadeTrue=entTrue[0]
                #print('textoEntidadeTrue:', textoEntidadeTrue)
                if textoEntidade == textoEntidadeTrue:
                    #print('temmm:', textoEntidade)
                    tem=1
                    labelEntidadeTrue = dicLabels[entTrue[2]]
                    #labelEntidadeTrue = entTrue[2]
                    regiao_true.append(labelEntidadeTrue)
                    regiao_pred.append(labelEntidade)
            if tem==0: # falsos positivos
                print('nao achou:', textoEntidade)
                print('labelEntidadeTrue:', labelEntidadeTrue)
                print('labelEntidade:', labelEntidade)
                regiao_true.append(labelEntidadeTrue)
                regiao_pred.append(labelEntidade)
                
        for entTrue in listaTrue:
            #print('entidade:', entidade)
            #print('numEnt:', numEnt)
            #print('label[numEnt]:', label[numEnt])
            textoEntidadeTrue=entTrue[0]
            #print('==textoEntidadeTrue:', textoEntidadeTrue)
            #labelEntidade=str(label[numEnt])
            #labelEntidade=dicLabels[label[numEnt]]
            #labelEntidade='0'
            #labelEntidadeTrue='Problema'
            labelEntidadeTrue=dicLabels[entTrue[2]]        
            tem=0
            for numEnt, entidade in enumerate(ent):
                textoEntidade=entidade[0]
                #print('textoEntidade:', textoEntidade)
                if textoEntidade == textoEntidadeTrue:
                    #print('temmm:', textoEntidade)
                    tem=1
                    break
            if tem==0:
                labelEntidade = '4'
                print('--nao achou222--:', textoEntidadeTrue)
                print('labelEntidadeTrue:', labelEntidadeTrue)
                print('labelEntidade:', labelEntidade)
                regiao_true.append(labelEntidadeTrue)
                regiao_pred.append(labelEntidade)
                
              
    #if num>40:
    #    break
print('len(regiao_pred):', len(regiao_pred))
print('len(regiao_true):', len(regiao_true))

print('regiao_pred[:14]:', regiao_pred[:14])
print('regiao_true[:14]:', regiao_true[:14])

print(flat_classification_report(
    regiao_pred, regiao_true, labels=['0','1','2','3','4'], digits=3
    #regiao_pred, regiao_true, labels=getLabels, digits=3
))

--nao achou222--: acarbose
labelEntidadeTrue: 1
labelEntidade: 4
--nao achou222--: digoxina
labelEntidadeTrue: 1
labelEntidade: 4
--nao achou222--: marevan
labelEntidadeTrue: 1
labelEntidade: 4
--nao achou222--: fc
labelEntidadeTrue: 2
labelEntidade: 4
--nao achou222--: BC arritmicas
labelEntidadeTrue: 0
labelEntidade: 4
--nao achou222--: panturrilhas
labelEntidadeTrue: 3
labelEntidade: 4
--nao achou222--: ureia
labelEntidadeTrue: 2
labelEntidade: 4
--nao achou222--: CPK
labelEntidadeTrue: 2
labelEntidade: 4
--nao achou222--: Triglerideos
labelEntidadeTrue: 2
labelEntidade: 4
--nao achou222--: HDL
labelEntidadeTrue: 2
labelEntidade: 4
--nao achou222--: CT
labelEntidadeTrue: 2
labelEntidade: 4
--nao achou222--: LDL
labelEntidadeTrue: 2
labelEntidade: 4
--nao achou222--: K
labelEntidadeTrue: 2
labelEntidade: 4
--nao achou222--: RNI
labelEntidadeTrue: 2
labelEntidade: 4
--nao achou222--: Microalbuminúria
labelEntidadeTrue: 2
labelEntidade: 4
--nao achou222--: ventrículo esquerdo com hiper

labelEntidade: 1
nao achou: Medicamentos atenolol
labelEntidadeTrue: 4
labelEntidade: 1
--nao achou222--: Medicamentos
labelEntidadeTrue: 1
labelEntidade: 4
nao achou: ablações
labelEntidadeTrue: 4
labelEntidade: 1
--nao achou222--: 02 ablações
labelEntidadeTrue: 1
labelEntidade: 4
--nao achou222--: dor de cabeça
labelEntidadeTrue: 0
labelEntidade: 4
--nao achou222--: cabeça
labelEntidadeTrue: 3
labelEntidade: 4
--nao achou222--: melhora com analgésico comum
labelEntidadeTrue: 0
labelEntidade: 4
--nao achou222--: analgésico comum
labelEntidadeTrue: 1
labelEntidade: 4
--nao achou222--: mmi
labelEntidadeTrue: 3
labelEntidade: 4
--nao achou222--: cardiaca
labelEntidadeTrue: 3
labelEntidade: 4
--nao achou222--: cisticercose
labelEntidadeTrue: 0
labelEntidade: 4
nao achou: Penicilina G benzatina 1 , 200 , 000U IM
labelEntidadeTrue: 4
labelEntidade: 1
--nao achou222--: Penicilina G benzatina 1 , 200 , 000U
labelEntidadeTrue: 1
labelEntidade: 4
nao achou: DOR REGIAO ESTERNAL , EM FO , LOCAL F

nao achou: PARA 100 MCG
labelEntidadeTrue: 4
labelEntidade: 1
nao achou: SINVASTATINA
labelEntidadeTrue: 4
labelEntidade: 1
--nao achou222--: FUROSEMIDA
labelEntidadeTrue: 1
labelEntidade: 4
--nao achou222--: FALTA DE AR
labelEntidadeTrue: 0
labelEntidade: 4
--nao achou222--: EXAMES
labelEntidadeTrue: 2
labelEntidade: 4
--nao achou222--: TSH
labelEntidadeTrue: 2
labelEntidade: 4
--nao achou222--: T4 LIVRE
labelEntidadeTrue: 2
labelEntidade: 4
--nao achou222--: CK
labelEntidadeTrue: 2
labelEntidade: 4
--nao achou222--: TGO
labelEntidadeTrue: 2
labelEntidade: 4
--nao achou222--: TGP
labelEntidadeTrue: 2
labelEntidade: 4
--nao achou222--: PERFIL LIPIDICO
labelEntidadeTrue: 2
labelEntidade: 4
--nao achou222--: MÃOS
labelEntidadeTrue: 3
labelEntidade: 4
nao achou: CINTILOGRAFIA MIOCÁRDICA
labelEntidadeTrue: 4
labelEntidade: 2
nao achou: DIPIRIDAMOL
labelEntidadeTrue: 4
labelEntidade: 2
nao achou: DO SEGMENTO ANTERIOR ( TRANSITÓRIO ) E BASAL DO SEGMENTO ANTERO - SEPTAL
labelEntidadeTrue: 4
l

C:\Users\lisat\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=['0', '1', '2', '3', '4'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [123]:
regiao_true[:10]

['0', '1', '0', '0', '1', '1', '1', '1', '0', '1']

In [124]:
regiao_pred[:10]

['0', '1', '0', '0', '1', '1', '1', '4', '0', '1']

In [84]:
listaFinal = list()
for labels, frase in zip(y_pred, listaPredictFrases):
    for label, entidade in zip(labels, frase):
        listaFinal.append([entidade, label])

In [86]:
listaFinal

[[['FA',
   'Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .'],
  'Problema'],
 [['marevan 5mg',
   'Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .'],
  'Tratamento'],
 [['Comorbidades',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'],
  'Problema'],
 [['DM',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'],
  'Problema'],
 [['metformina 850mg',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'],
  'Tratamento'],
 [['glicazida 60mg',
   'Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .'],
  'O'],
 [['insulina',
   'C

In [93]:
def sent2tokens(sent):
    return [token for token, postag in sent]


In [ ]:
#def getListaPositiva(dic_sentences):
#dic_sentences = dic_sentencesTest
'''
def geraListaPredict(frases):
    for frase in frases:
    numJanela=4
    
    
    
    
    listaSentencas=list()
    lista=list()
    dicPred={}
    #numJanela=NUM_JANELA
    num1=-1
    num2=0
    for key, value in dic_sentences.items():
        num1=num1+1
        #print('value:', value)
        entidades = value[1]
        if len(entidades)==0:
            dicPred[num2]=num1
            continue
        dicPred[num2]=num1
        num2=num2+1
        listaIndicesEntidadesFrase=list()
        for entidade in entidades:
          listaIndicesEntidadesFrase.append(entidade[1])
        dicTokens={}
        for token in value[0]:
          #print('token:', token)
          dicTokens[token[1]]=token[0]
        for entidade in entidades:
            #print('entidade:', entidade)
            label = entidade[2]
            #label = 'ENT'
            indiceEntidade1=entidade[1][0]
            indiceEntidade2=entidade[1][-1]
            vizinhosAntes = list()
            vizinhosDepois = list()
            #print('indiceEntidade:', indiceEntidade)
            for tokens in value[0]:
                indice=tokens[1]
                #print('token: {}, indice: {}'.format(tokens[0], indice))
                if indice+numJanela>=indiceEntidade1 and indice<indiceEntidade1:
                    vizinhosAntes.append(tokens[0])
                if indice-numJanela<=indiceEntidade2 and indice>indiceEntidade2:
                    vizinhosDepois.append(tokens[0])
            lista.append([entidade[0], vizinhosAntes, vizinhosDepois, label])
            # agora, os negativos, para entidades com mais de um token
            if len(entidade[0].split())>1:
              #print('entidade[0] mais de um token: {}'.format(entidade[0].split()))
              #listaCombinacoes = powerset(entidade[0].split())
              listaCombinacoes = powerset(entidade[1])
              #print(listaCombinacoes)
              # ver se alguma dessas é positiva
              for combinacao in listaCombinacoes:
                #print('combinacao:', combinacao)
                if combinacao in listaIndicesEntidadesFrase:
                  #print('teeeem:', combinacao)
                  pass
                else:
                  combinacaoTokens = [dicTokens[t] for t in combinacao]
                  vizinhosAntes = list()
                  vizinhosDepois = list()
                  indiceEntidade1=combinacao[0]
                  indiceEntidade2=combinacao[-1]
                  for tokens in value[0]:
                    indice=tokens[1]
                    #print('token: {}, indice: {}'.format(tokens[0], indice))
                    if indice+numJanela>=indiceEntidade1 and indice<indiceEntidade1:
                        vizinhosAntes.append(tokens[0])
                    if indice-numJanela<=indiceEntidade2 and indice>indiceEntidade2:
                        vizinhosDepois.append(tokens[0])
                  lista.append([' '.join(combinacaoTokens), vizinhosAntes, vizinhosDepois, 'O'])
        listaSentencas.append(lista)
        lista=list()
        #break
    return listaSentencas, dicPred

listaTeste=geraListaPredict(['oi, tudo bem?'])
'''

In [121]:
def checkConsecutive(l):
    return sorted(l) == list(range(min(l), max(l)+1))
      
# Driver Code
lst = [2, 3, 1, 4, 5]
#print(checkConsecutive(lst))

for key, value in dicSentences_new_test.items():
    print(key)
    for ent in value[1]:
        print(ent)
        indice=ent[1]
        if not checkConsecutive(indice):
            print(key)
    #break

0
1
['FA', [8], 'Problema']
['marevan 5mg', [12, 13], 'Tratamento']
2
['Comorbidades', [0], 'Problema']
['DM', [2], 'Problema']
['metformina 850mg', [9, 10], 'Tratamento']
['acarbose', [16], 'Tratamento']
['glicazida 60mg', [22, 23], 'Tratamento']
['insulina', [29], 'Tratamento']
3
['HAS', [0], 'Problema']
['losartana 50mg', [7, 8], 'Tratamento']
['digoxina', [12], 'Tratamento']
['carvedilol 25', [20, 21], 'Tratamento']
['HCTZ', [26], 'Tratamento']
4
['DSLP', [0], 'Problema']
['sinvastatina', [4], 'Tratamento']
['marevan', [6], 'Tratamento']
5
6
['dispnéia', [1], 'Problema']
['DPN', [3], 'Problema']
['ortopneia', [5], 'Problema']
['palpitações', [7], 'Problema']
['sincope', [9], 'Problema']
7
['afebril', [8], 'Problema']
['fc', [10], 'Teste']
['so2', [13], 'Teste']
['pa', [17], 'Teste']
['RA', [28], 'Problema']
8
['BC arritmicas', [0, 1], 'Problema']
['SS', [4], 'Problema']
9
['Abd', [0], 'Anatomia']
['VCM', [10], 'Problema']
10
['MMII', [0], 'Anatomia']
['edema', [2], 'Problema']
['pa

['arritmias - terapias', [4, 5, 6], 'Tratamento']
403
404
['atrial', [2], 'Anatomia']
['ventricular', [4], 'Anatomia']
405
406
407
['CAT', [1], 'Teste']
408
['VALVOPLASTIA AÓRTICA', [0, 1], 'Tratamento']
['AÓRTICA', [1], 'Anatomia']
409
['VALVA BIOLÓGICA', [0, 1], 'Tratamento']
['CX', [7], 'Tratamento']
['CARDIOVERSÃO', [8], 'Tratamento']
410
['TONTURA', [24], 'Problema']
411
412
['QUEIXAS', [6], 'Problema']
413
['EMAGRECIMENTO DE 7 KG', [1, 2, 3, 4], 'Problema']
414
['EX - TABAGISTA', [1, 2, 3], 'Problema']
415
['ENALAPRIL 10MG', [5, 6], 'Tratamento']
['CARVEDILOL 12 , 5 MG', [11, 12, 13, 14, 15], 'Tratamento']
['ATORVASTATINA 40 MG', [20, 21, 22], 'Tratamento']
['CLOPIDOGREL', [24], 'Tratamento']
['MAREVAN', [26], 'Tratamento']
416
['EXAME FÍSICO', [1, 2], 'Teste']
417
418
['FC', [0], 'Teste']
['PA', [4], 'Teste']
419
['HIPOFONESE DE BULHAS EM FOCO AÓRTICO', [2, 3, 4, 5, 6, 7], 'Problema']
['FOCO AÓRTICO', [6, 7], 'Anatomia']
420
['RA', [7], 'Problema']
421
['ABDOME', [3], 'Anatomia'

In [104]:
#listaTeste=geraListaPredict()
listaTeste = [
    [['FA', ['há', '5', 'anos', 'por'], [',', 'uso', 'de', 'marevan'], 'Problema'], ['marevan 5mg', ['FA', ',', 'uso', 'de'], ['1', 'x', 'ao', 'dia'], 'Tratamento'], ['marevan', ['FA', ',', 'uso', 'de'], ['5mg', '1', 'x', 'ao'], 'O'], ['5mg', [',', 'uso', 'de', 'marevan'], ['1', 'x', 'ao', 'dia'], 'O']]

]
for i in range(0,5):
    example_sent = listaTeste[i]
    print("\n")
    print(example_sent)
    #print("Sentence:", ' '.join(sent2tokens(example_sent)))
    print("Predicted:", ' '.join(crf.predict([sent2features(example_sent, word2cluster, dicPostagger)])[0]))
    print("Correct:  ", ' '.join(sent2labels(example_sent)))



[['FA', ['há', '5', 'anos', 'por'], [',', 'uso', 'de', 'marevan'], 'Problema'], ['marevan 5mg', ['FA', ',', 'uso', 'de'], ['1', 'x', 'ao', 'dia'], 'Tratamento'], ['marevan', ['FA', ',', 'uso', 'de'], ['5mg', '1', 'x', 'ao'], 'O'], ['5mg', [',', 'uso', 'de', 'marevan'], ['1', 'x', 'ao', 'dia'], 'O']]
Predicted: Problema Tratamento O O
Correct:   Problema Tratamento O O


IndexError: list index out of range